In [1]:
%load_ext autoreload
%autoreload 2

from itertools import combinations
import os

import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import seaborn as sns

import helpers as hlp

### Define CSV and read in data

**TODO - read in both 2021 and 2022 files and concatenate**

In [2]:
# Define file
csv_file_21 = 'dc_311_2021_data.csv'
csv_file_22 = 'dc_311_2022_data.csv'

In [3]:
# Read in data
data_dir = os.path.join('..', '..', 'data', 'raw')

df_21 = pd.read_csv(os.path.join(data_dir, csv_file_21))
df_21['api_year'] = 2021

df_22 = pd.read_csv(os.path.join(data_dir, csv_file_22))
df_22['api_year'] = 2022

df = pd.concat([df_21, df_22], ignore_index=True)

/var/folders/rq/rs89ll8n2w93x8gnl0mtn7sw0000gn/T/ipykernel_62395/2072760993.py:4: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_21 = pd.read_csv(os.path.join(data_dir, csv_file_21))
/var/folders/rq/rs89ll8n2w93x8gnl0mtn7sw0000gn/T/ipykernel_62395/2072760993.py:7: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_22 = pd.read_csv(os.path.join(data_dir, csv_file_22))


### Transform data to make EDA easier

In [4]:
# Convert all column names to lowercase
df.columns = [col.lower() for col in df.columns]

In [5]:
# Convert columns to date/time as appropriate
time_cols = ['adddate', 'resolutiondate', 'serviceduedate', 'serviceorderdate',
             'inspectiondate', 'gdb_from_date', 'gdb_to_date']
for col in time_cols:
    df[col] = pd.to_datetime(df[col], unit='ms', errors='coerce')

### Profile dataframe

Initial observations:<br>
- `objectid`: Confirmed that all values are unique.
- `servicecode`: Maps basically 1:1 with `servicecodedescription`. Seven service codes have 2 descriptions, but all descriptions appear to be describing the same thing.
- `servicecodedescription`: See `servicecode`. We can DROP from as a feature, as all information is included in `servicecode`. Or we can replace the service codes with the description.
- `servicetypecodedescription`: Maps closely to `organizationacronym`. We can DROP as a feature for now (but maybe bring back later).
- `organizationacronym`: KEEP as a feature.
- `servicecallcount`: DROP. All values are 1.
- `addate`:
- `resolutiondate`:
- `serviceduedate`:
- `serviceorderdate`:
- `serviceorderstatus`:
- `status_code`:
- `servicerequestid`:
- `priority`:
- `streetaddress`:
- `xcoord`:
- `ycoord`:
- `latitude`:
- `longitude`:
- `city`:
- `state`:
- `zipcode`
- `maraddressrepositoryid`:
- `ward`:
- `details`:
- `gis_id`:
- `globalid`:
- `creator`:
- `created`:
- `editor`:
- `edited`:
- `gdb_from_date`:
- `gdb_to_date`:
- `gdb_archive_oid`:
- `api_year`:

##### Service Codes

In [6]:
service_code_cols = ["servicecode", "servicecodedescription",
                    "servicetypecodedescription", "organizationacronym"]
code_df = df[service_code_cols]

In [7]:
col_pairs = list(combinations(list(code_df.columns), 2))
for pair in col_pairs:
    hlp.check_if_two_fields_are_one_to_one(code_df, pair[0], pair[1])
    print("-------------------------------------------------------------------")
    print("-------------------------------------------------------------------")


There are 7 values of `servicecode` that have more than one `servicecodedescription`:
servicecode
C62313m       2
ILLEGALCON    2
INSECTT01     2
OUTODOR       2
S0196         2
SRC02         2
VACPROP       2
Name: servicecodedescription, dtype: int64
----------------------------------------------------
All values of `servicecodedescription` have at most one `servicecode`.
-------------------------------------------------------------------
-------------------------------------------------------------------
There are 2 values of `servicecode` that have more than one `servicetypecodedescription`:
servicecode
ILLEGALCON    2
VACPROP       2
Name: servicetypecodedescription, dtype: int64
----------------------------------------------------
There are 20 values of `servicetypecodedescription` that have more than one `servicecode`:
servicetypecodedescription
311- Call Center                                        6
311- Emergencies                                        2
Adjudication Servic

In [8]:
code_df.head()

,servicecode,servicecodedescription,servicetypecodedescription,organizationacronym
0,S0031,Bulk Collection,SWMA- Solid Waste Management Admistration,DPW
1,BICYCLE,Abandoned Bicycle,SWMA- Solid Waste Management Admistration,DPW
2,S05SL,Streetlight Repair Investigation,Transportation Operations Administration,DDOT
3,S0031,Bulk Collection,SWMA- Solid Waste Management Admistration,DPW
4,S05SL,Streetlight Repair Investigation,Transportation Operations Administration,DDOT


All service codes have just one service code description, except for:<br>
`C62313m`, `ILLEGALCON`, `OUTODOR`, `INSECTT01`, `SRC02`, `VACPROP`, `S0196`.<br>

(Based on the below, it looks like the multiple code descriptions are rewordings of the same request type.)

All service code descriptions are assigned to only one service code.

In [9]:
codes = ["C62313m", 'ILLEGALCON', "OUTODOR", "INSECTT01", "SRC02", "VACPROP", "S0196"]
for code in codes:
    print("-------------------------------------------------")
    print(f"Service code is: {code}")
    code_desc_df = code_df[code_df['servicecode'] == code]['servicecodedescription'].value_counts()
    print(code_desc_df)


-------------------------------------------------
Service code is: C62313m
servicecodedescription
Christmas Tree Removal-Missed      1981
Christmas Tree Removal - Missed       1
Name: count, dtype: int64
-------------------------------------------------
Service code is: ILLEGALCON
servicecodedescription
DCRA - Illegal Construction    2174
DOB - Illegal Construction      330
Name: count, dtype: int64
-------------------------------------------------
Service code is: OUTODOR
servicecodedescription
DOEE - General Air Quality Concerns    241
DOEE - Nuisance Odor Complaints         52
Name: count, dtype: int64
-------------------------------------------------
Service code is: INSECTT01
servicecodedescription
Insect Treatment                         628
Bee Treatment or Arboviral Inspection    275
Name: count, dtype: int64
-------------------------------------------------
Service code is: SRC02
servicecodedescription
Snow/Ice Removal                      2123
Snow/Ice Removal on Public Stree

Is the `servicetypecodedescription` and `organizationacronym` relationship one-to-one?

_Each service type code description is only assigned one organization acronym._<br>
_Each organization is assigned to multiple service type codes. So organization is giving some order to service type code description._

In [10]:
orgs = ["DDOT", 'DMV', "DPW", "FEMS", "OUC"]
for org in orgs:
    print("-------------------------------------------------")
    print(f"Org is: {org}")
    org_df = code_df[code_df['organizationacronym'] == org]['servicetypecodedescription'].value_counts()
    print(org_df)

-------------------------------------------------
Org is: DDOT
servicetypecodedescription
Transportation Operations Administration           53350
Urban Forrestry                                    31600
Toa-Trans Sys Mnt-Signs                            21575
Toa-Street & Bridge Maintenance                    16019
Department of Transportation                        8312
Toa- Trans Sys Mnt                                  8169
SIOD                                                6804
Transportation Policy & Planning Administration     1807
Name: count, dtype: int64
-------------------------------------------------
Org is: DMV
servicetypecodedescription
Driver Vehicle Services    15054
Adjudication Services       1476
Name: count, dtype: int64
-------------------------------------------------
Org is: DPW
servicetypecodedescription
SWMA- Solid Waste Management Admistration              318963
PEMA- Parking Enforcement Management Administration    156300
SWMA- Solid Waste Management Admin

##### Dates

Observations:<br>
- `adddate`: All records have an add date. Min and max are in expected years.
- `resolutiondate`:
- `serviceduedate`:
- `serviceorderdate`:

##### Understand `Days to Resolve` Causes

##### `Days to Resolve` Initial Review

In [11]:
date_cols = ["servicecodedescription", "organizationacronym", "adddate",
            "resolutiondate", "serviceduedate", "serviceorderdate"]
date_df = df[date_cols]
date_df['days_to_resolve'] = (date_df['resolutiondate'] - date_df['adddate']).dt.days

/var/folders/rq/rs89ll8n2w93x8gnl0mtn7sw0000gn/T/ipykernel_62395/2810958849.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_df['days_to_resolve'] = (date_df['resolutiondate'] - date_df['adddate']).dt.days


In [12]:
date_df.shape

(747490, 7)

~75% of 311 requests are completed within a week. Average time to complete is 25 days, versus a median of 3 days. Definitely a right-skewed distribution.

If I'm someone calling 311, I may not be interested in days to resolve. What I care about most: what's the probability my request is an "easy" one, or a difficult one? Maybe: probability request completed in <5 days and request completed in >3 weeks?

In [13]:
date_df.describe()

,adddate,resolutiondate,serviceduedate,serviceorderdate,days_to_resolve
count,747490,740138,747243,747490,740138.000000
mean,2022-01-12 18:45:28.402591232,2022-02-07 04:18:48.196868096,2022-02-18 18:51:51.969645824,2022-01-12 18:45:28.402591232,25.803947
min,2021-01-01 05:06:02,2021-01-01 05:07:01,2021-01-03 20:29:16,2021-01-01 05:06:02,-1.000000
25%,2021-07-26 19:32:42.500000,2021-08-11 17:03:59,2021-08-20 03:59:00,2021-07-26 19:32:42.500000,0.000000
50%,2022-01-11 22:09:28,2022-02-09 13:16:55.500000,2022-02-16 13:21:11,2022-01-11 22:09:28,3.000000
75%,2022-07-10 21:14:29.750000128,2022-07-28 16:27:31.750000128,2022-08-11 21:37:41,2022-07-10 21:14:29.750000128,8.000000
max,2023-01-01 04:46:01,2024-09-25 21:03:09,2024-12-18 19:39:51,2023-01-01 04:46:01,1348.000000
std,NaN,NaN,NaN,NaN,92.773416


In [14]:
percentile_list = [0.01, 0.1, 0.25, 0.5, 0.6, 0.75, 0.85, 0.92, 0.95, 0.99, 0.999]
date_df['days_to_resolve'].quantile(percentile_list)

0.010       0.0
0.100       0.0
0.250       0.0
0.500       3.0
0.600       5.0
0.750       8.0
0.850      18.0
0.920      63.0
0.950     139.0
0.990     522.0
0.999    1014.0
Name: days_to_resolve, dtype: float64

##### `Days to Resolve` by Service Code

In [34]:
df['days_to_resolve'] = (date_df['resolutiondate'] - date_df['adddate']).dt.days

In [56]:
agg_servicecode_df = hlp.summarize_relationship_between_target_and_variable(
    df, 'servicecode', 'servicecodedescription'
)
agg_servicecode_df.sort_values('count_records', ascending=False).head(15)

,servicecode,median_days_to_resolve,std_dev_days_to_resolve,count_records,servicecodedescription
149980,S0031,5.0,2.286277,137637,Bulk Collection
336638,S0261,0.0,3.455315,94977,Parking Enforcement
575369,S0441,1.0,3.503291,34223,Trash Collection - Missed
291151,S0181,6.0,13.197451,28062,Illegal Dumping
431615,S0276,2.0,45.129480,26300,Parking Meter Repair
464808,S0311,1.0,9.880734,25686,Rodent Inspection and Treatment
670695,S05SL,3.0,28.679616,23831,Streetlight Repair Investigation
90309,RPP,0.0,1.241851,23086,Residential Parking Permit Violation
113395,S0000,6.0,76.498590,22979,Abandoned Vehicle - On Public Property
642434,S0477,7.0,3.988949,22769,Scheduled Yard Waste


`Days to Resolve` by Ward

There is some spread, but the medians are basically the same. Interestingly, the 8th Ward, which has the lowest median household income of the wards, has the lowest average time to resolve

In [62]:
agg_ward_df = hlp.summarize_relationship_between_target_and_variable(
    df, 'ward'
)
agg_ward_df.sort_values('ward', ascending=True).head(30)

,ward,median_days_to_resolve,std_dev_days_to_resolve,avg_days_to_resolve,count_records
0,1,3.0,72.240443,20.690026,92242
1,2,2.0,93.423146,28.286897,83488
2,3,4.0,102.684604,30.763829,62169
3,4,4.0,113.724130,33.806471,103823
4,5,4.0,104.603820,29.824726,112264
5,6,3.0,79.697266,21.253380,112291
6,7,4.0,90.421910,23.007805,96348
7,8,2.0,71.187549,17.737688,80566
8,Null,5.0,134.526849,59.947564,3552
9,Ward 1,14.0,152.472948,98.000000,3


##### Understand Unresolved Cases

Unresolved cases don't generally seem to be instances of bad data. Generally, < 0.5% of a given service code is unresolved, but there are a small number of service codes where the percent of unresolved cases spikes, indicating that these are actually unresolved cases and not just accidental missing data.

In [28]:
unresolved_df = df[date_df['resolutiondate'].isnull()]

In [29]:
unresolved_df[['adddate']].describe()

,adddate
count,7352
mean,2022-04-08 13:38:43.269178368
min,2021-01-02 19:37:57
25%,2021-10-13 21:33:18
50%,2022-06-07 17:18:02.500000
75%,2022-09-07 14:20:41.750000128
max,2022-12-31 16:21:58


~30% of unresolved cases are from 2021, remainder from 2022

In [30]:
percentile_list = [0, 0.01, 0.05, 0.1, 0.25, 0.3, 0.5, 0.7, 0.75, 0.9, 0.95, 0.99, 1]
unresolved_df['adddate'].quantile(percentile_list)

0.00   2021-01-02 19:37:57.000000000
0.01   2021-01-28 06:04:08.910000128
0.05   2021-03-22 18:58:58.450000128
0.10   2021-06-03 17:59:48.200000000
0.25   2021-10-13 21:33:18.000000000
0.30   2022-01-24 04:29:15.400000000
0.50   2022-06-07 17:18:02.500000000
0.70   2022-09-02 16:10:23.600000000
0.75   2022-09-07 14:20:41.750000128
0.90   2022-10-19 17:50:14.900000000
0.95   2022-11-14 16:08:36.249999872
0.99   2022-12-17 15:56:17.069999872
1.00   2022-12-31 16:21:58.000000000
Name: adddate, dtype: datetime64[ns]

Requests for repair seem to make up at least ~40% of unresolved cases and 6 of top 10 most-frequently-unresolved types of cases

In [31]:
unresolved_freq_df = pd.DataFrame(unresolved_df['servicecode'].value_counts()).reset_index()
service_code_freq_df = pd.DataFrame(df['servicecode'].value_counts()).reset_index()

merged_freq_df = unresolved_freq_df.merge(service_code_freq_df, on="servicecode", suffixes=('_unresolved', '_all'))
merged_freq_df['pct_unresolved'] = (merged_freq_df['count_unresolved'] /
                                    merged_freq_df['count_all'])
merged_freq_df['pct_of_all_unresolved']= (merged_freq_df['count_unresolved']/
                                          sum(merged_freq_df['count_unresolved']))
merged_freq_df['pct_all'] = merged_freq_df['count_all'] / sum(merged_freq_df['count_all'])


In [32]:
description_df = df[['servicecode', 'servicecodedescription']]
description_df = description_df.drop_duplicates(['servicecodedescription'])
merged_freq_df = merged_freq_df.merge(description_df, on='servicecode', how='left')

In [33]:
display(merged_freq_df.sort_values('pct_of_all_unresolved', ascending=False))

,servicecode,count_unresolved,count_all,pct_unresolved,pct_of_all_unresolved,pct_all,servicecodedescription
0,S0361,1355,9126,0.148477,0.184304,0.013087,Sidewalk Repair
1,INSECTT01,882,903,0.976744,0.119967,0.001295,Insect Treatment
2,INSECTT01,882,903,0.976744,0.119967,0.001295,Bee Treatment or Arboviral Inspection
3,S0406,661,2775,0.238198,0.089908,0.003979,Roadway Repair
4,S0016,526,1581,0.332701,0.071545,0.002267,Alley Repair
5,S0031,523,137637,0.003800,0.071137,0.197374,Bulk Collection
6,S0423,332,2485,0.133602,0.045158,0.003564,Supercan - Repair
7,TRACO001,294,2937,0.100102,0.039989,0.004212,Trash Cart Repair
8,MARKINST,243,3219,0.075489,0.033052,0.004616,Roadway Striping / Markings
9,LCanRep,156,165,0.945455,0.021219,0.000237,Public Space Litter Can Repair
